<a href="https://colab.research.google.com/github/RogerHeederer/DeepLearning_Heo/blob/master/RNN_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference source : https://github.com/minsuk-heo/tf2 <br/>
공부 과정에서 필요한 부분에 대해서는 code, figure 수정, 삽입 등을 수행했습니다.

In [5]:
from IPython.display import Image

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorlfow_version 2.x
except Exception:
  pass

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN, TimeDistributed
from tensorflow.keras.models import Model, Sequential
import numpy as np

##RNN CELL 분석

In [8]:
inputs = Input(shape=(1,2))

In [9]:
inputs

<tf.Tensor 'input_1:0' shape=(None, 1, 2) dtype=float32>

In [10]:
output, state = SimpleRNN(3, return_state=True, activation='tanh')(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [11]:
output, state, model

(<tf.Tensor 'simple_rnn/strided_slice_3:0' shape=(None, 3) dtype=float32>,
 <tf.Tensor 'simple_rnn/while:4' shape=(None, 3) dtype=float32>,
 <tensorflow.python.keras.engine.functional.Functional at 0x7fab1a623cc0>)

In [12]:
model.layers[0], model.layers[1] #SimpleRNN

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7fab4dde67b8>,
 <tensorflow.python.keras.layers.recurrent.SimpleRNN at 0x7fab1a623748>)

In [13]:
data = np.array([[ [1,2] ]])

In [14]:
data, data.shape

(array([[[1, 2]]]), (1, 1, 2))

In [15]:
output, state = model.predict(data)

In [16]:
output, state # 아웃풋과 현재 상태값은 같은 것을 확인할 수 있다.

(array([[ 0.08964063,  0.830997  , -0.92504376]], dtype=float32),
 array([[ 0.08964063,  0.830997  , -0.92504376]], dtype=float32))

In [17]:
model.layers[1].weights[0] #input shape (1,2) X (2,3) weights_shape = (1,3) = output and state

<tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[ 0.5427234 , -0.04074979, -0.2556253 ],
       [-0.22642076,  0.61604977, -0.68363726]], dtype=float32)>

In [18]:
model.layers[1].weights[1] # prior state shape (1,3) X (3,3) weights_shape = (1,3) = output and state

<tf.Variable 'simple_rnn/simple_rnn_cell/recurrent_kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.9243765 , -0.22979698, -0.30450234],
       [-0.38039708,  0.61539215,  0.69035536],
       [ 0.02874677,  0.75398   , -0.65626806]], dtype=float32)>

In [19]:
model.layers[1].weights[2]

<tf.Variable 'simple_rnn/simple_rnn_cell/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

##Sequence tagging example

In [20]:
John = [1,0,0]
loves = [0,1,0]
Jane = [0,0,1]

X = np.array([
  [John, loves, Jane],
  [Jane, loves, John]
]).astype(np.float32)

In [21]:
X

array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]]], dtype=float32)

In [22]:
S = [0]
V = [1]
O = [2]
y = np.array([[S, V, O], [S, V, O]]).astype(np.float32)

In [23]:
y

array([[[0.],
        [1.],
        [2.]],

       [[0.],
        [1.],
        [2.]]], dtype=float32)

In [24]:
inputs = Input(shape=(3,3))

In [25]:
output, state = SimpleRNN(3, return_state=True, return_sequences=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [26]:
output, state = model.predict(X)

In [27]:
print("John loves Jane: ",output[0])
print("Jane loves John: ",output[1])

John loves Jane:  [[ 0.24743809 -0.6455617   0.69018984]
 [ 0.45676604 -0.8584785   0.87062585]
 [-0.19132279  0.25399846  0.9211307 ]]
Jane loves John:  [[ 0.0876617   0.6959274   0.43969262]
 [ 0.30781063 -0.12050796  0.55869275]
 [-0.05357769 -0.6460992   0.88973737]]


In [28]:
# state 값은 각 문장의 마지막 output 값과 일치하는 것을 확인
print("John loves Jane: state: ",state[0])
print("Jane loves John: state: ",state[1])

John loves Jane: state:  [-0.19132279  0.25399846  0.9211307 ]
Jane loves John: state:  [-0.05357769 -0.6460992   0.88973737]


<img src='https://drive.google.com/uc?export=view&id=1nNqmFJUc_u-blPWXCohEzUNCxS_7oAr7' align="left" width=600>

In [29]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(3, 3), return_sequences=True))
model.add(TimeDistributed(Dense(3, activation="softmax")))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 3, 3)              21        
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 3)              12        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
model.fit(X, y, epochs=2000, verbose=0)

In [56]:
result = model.predict(X, verbose=0)

In [57]:
result

array([[[9.9894983e-01, 7.0501858e-04, 3.4522399e-04],
        [6.9352245e-04, 9.9916077e-01, 1.4562858e-04],
        [6.4191496e-04, 2.2367523e-04, 9.9913448e-01]],

       [[9.9899596e-01, 6.8311434e-04, 3.2094790e-04],
        [7.8073418e-04, 9.9907541e-01, 1.4383254e-04],
        [6.2991871e-04, 2.2896664e-04, 9.9914110e-01]]], dtype=float32)

In [58]:
np.argmax(result, axis=1)

array([[0, 1, 2],
       [0, 1, 2]])